<a href="https://colab.research.google.com/github/gireeshdeepakr/dezoomcamp2024/blob/main/cohorts/2024/workshops/Homework_data_talks_club_data_extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [68]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
total = 0
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value);
    total += sqrt_value;
print(f'Sum of the outputs of the generator for limit = 5 is:',total)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
Sum of the outputs of the generator for limit = 5 is: 8.382332347441762


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [69]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

first_generator_sum_of_ages = 0
for person in people_1():
    print(person)
    #print(person["Age"])
    first_generator_sum_of_ages += person["Age"]
print(f'First generator sum of ages:',first_generator_sum_of_ages)



def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}
second_generator_sum_of_ages = first_generator_sum_of_ages

for person in people_2():
    print(person)
    second_generator_sum_of_ages += person["Age"]
print(f'Second generator sum of ages after append:',second_generator_sum_of_ages)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
First generator sum of ages: 140
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
Second generator sum of ages after append: 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [70]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [71]:
import duckdb
import dlt
#import pandas as pd

# Initialize DuckDB
#con = duckdb.connect(database=':memory:', read_only=False)

pipeline = dlt.pipeline(destination="duckdb", dataset_name="dlt_workshop")

# Define the generators
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# Create table and load first generator, use merge disposition so that data will not be duplicated in the subscequent runs
info = pipeline.run(people_1(), table_name="people", write_disposition="merge", primary_key= "id")
# Load the second generator and merge into existng table which has first generator
info = pipeline.run(people_2(), table_name="people", write_disposition="merge", primary_key= "id")

# Print all rows in the people table and calculate the sum of ages
with pipeline.sql_client() as client:
    conn = client.native_connection
    res_1 = conn.sql('SELECT * FROM people ORDER BY ID');
    res_2 = conn.sql('SELECT SUM(Age) FROM people');
    res_1.show()
    res_2.show()


┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708197869.0608485 │ iaG6TLmWKDY0KA │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708197869.0608485 │ OfDGmZSh992vkg │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1708197870.3612876 │ 7xByK/a8AcM3Ew │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1708197870.3612876 │ yc1Bhf9xPPx6+Q │ Job_4      │
│     5 │ Person_5 │    35 │ City_B  │ 1708197870.3612876 │ F8ZjKAcV7838KQ │ Job_5      │
│     6 │ Person_6 │    36 │ City_B  │ 1708197870.3612876 │ un2AdyyyVZpihA │ Job_6      │
│     7 │ Person_7 │    37 │ City_B  │ 1708197870.3612876 │ EYsy64R208kRBQ │ Job_7      │
│     8 │ 

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX